In [1]:
# Cell 1: System Initialization
# ===================================
# This cell sets up PyNucleus and prepares all components

import sys
from pathlib import Path

print("🔧 Initializing PyNucleus Model...")

# Add src to Python path
src_path = str(Path().resolve() / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

try:
    # Import PyNucleus components
    from pynucleus.pipeline import PipelineUtils
    from pynucleus.integration.llm_output_generator import LLMOutputGenerator
    
    # Initialize pipeline
    pipeline = PipelineUtils(results_dir="data/05_output/results")
    llm_generator = LLMOutputGenerator(results_dir="data/05_output/reports")
    
    print("✅ PyNucleus Model initialized successfully!")
    print("📋 System Ready:")
    print("   • RAG Pipeline - Document processing and retrieval")
    print("   • DWSIM Pipeline - Chemical process simulation")
    print("   • Results Export - CSV and report generation")
    print("   • LLM Integration - Intelligent analysis and summaries")
    print("\n🎯 Ready to run analysis! Execute Cell 2 to start.")
    
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print("💡 Please ensure you're in the PyNucleus-Model directory")
except Exception as e:
    print(f"❌ Initialization Error: {e}")
    print("💡 Please check your system setup")


🔧 Initializing PyNucleus Model...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Setting up RAG imports...
✅ RAG imports ready!
📂 Loaded 5 existing DWSIM results from disk
🔧 Setting up DWSIM imports...
✅ DWSIM modules imported successfully
📁 Results directory: data/05_output/results
🔧 Pipeline Utils initialized with results dir: data/05_output/results
🔗 DWSIM-RAG integration enabled
✅ PyNucleus Model initialized successfully!
📋 System Ready:
   • RAG Pipeline - Document processing and retrieval
   • DWSIM Pipeline - Chemical process simulation
   • Results Export - CSV and report generation
   • LLM Integration - Intelligent analysis and summaries

🎯 Ready to run analysis! Execute Cell 2 to start.


In [2]:
# Cell 2: Run Complete Analysis
# ===================================
# This cell runs the complete PyNucleus pipeline

print("🚀 Starting Complete PyNucleus Analysis...")
print("\n📊 This will run:")
print("   1. Document processing and RAG analysis")
print("   2. DWSIM chemical process simulations")
print("   3. Results export and report generation")
print("\n⏳ Please wait... This may take 20-30 seconds.\n")

try:
    # Run the complete pipeline
    results = pipeline.run_complete_pipeline()
    
    if results:
        print(f"\n🎉 Analysis completed successfully in {results['duration']:.1f} seconds!")
        print("\n📊 Results Summary:")
        print(f"   • Documents Processed: {len(results['rag_data'])} queries")
        print(f"   • Simulations Completed: {len(results['dwsim_data'])} scenarios")
        print(f"   • Files Generated: {len(results['exported_files'])} CSV files")
        
        # Generate enhanced reports if available
        try:
            from pynucleus.integration.dwsim_rag_integrator import DWSIMRAGIntegrator
            
            integrator = DWSIMRAGIntegrator(
                rag_pipeline=pipeline.rag_pipeline,
                results_dir="data/05_output/results"
            )
            
            # Enhanced analysis
            dwsim_results = pipeline.dwsim_pipeline.get_results()
            if dwsim_results:
                enhanced_results = integrator.integrate_simulation_results(
                    dwsim_results, perform_rag_analysis=True
                )
                
                # Generate LLM reports
                report_files = []
                for result in enhanced_results[:3]:  # Generate reports for first 3 simulations
                    report_file = llm_generator.export_llm_ready_text(result)
                    report_files.append(report_file)
                
                # Financial analysis
                financial_file = llm_generator.export_financial_analysis(enhanced_results)
                metrics = llm_generator._calculate_key_metrics(enhanced_results)
                
                print("\n💰 Financial Analysis:")
                print(f"   • Recovery Rate: {metrics['avg_recovery']:.1f}%")
                print(f"   • Daily Revenue: ${metrics['estimated_revenue']:,.2f}")
                print(f"   • Daily Profit: ${metrics['net_profit']:,.2f}")
                print(f"   • ROI: {metrics['roi']:.1f}%")
                
                print(f"\n📄 Generated Reports: {len(report_files)} detailed analysis files")
                
        except Exception as e:
            print("⚠️ Enhanced analysis unavailable (using basic results only)")
        
        print("\n📁 All results saved to:")
        print("   • CSV Files: data/05_output/results/")
        print("   • Reports: data/05_output/reports/")
        print("\n✅ Analysis complete! Run Cell 3 to explore your results.")
        
    else:
        print("❌ Pipeline execution failed")
        print("💡 Please check your data directories and try again")
        
except Exception as e:
    print(f"❌ Error during analysis: {e}")
    print("💡 Please ensure all components are properly initialized")


🚀 Starting Complete PyNucleus Analysis...

📊 This will run:
   1. Document processing and RAG analysis
   2. DWSIM chemical process simulations
   3. Results export and report generation

⏳ Please wait... This may take 20-30 seconds.

🚀 Running complete PyNucleus pipeline...
🗑️ RAG results cleared.
🗑️ RAG results cleared.
🗑️ DWSIM results cleared from memory and disk.
🗑️ DWSIM results cleared.
🔬 Step 1: Running DWSIM simulations...
🔬 Starting DWSIM Simulations...
📋 Running 5 simulation cases...

🧪 Case 1/5: distillation_ethanol_water
   ✅ Success - Duration: 0.00s

🧪 Case 2/5: reactor_methane_combustion
   ✅ Success - Duration: 0.00s

🧪 Case 3/5: heat_exchanger_steam
   ✅ Success - Duration: 0.00s

🧪 Case 4/5: absorber_co2_capture
   ✅ Success - Duration: 0.00s

🧪 Case 5/5: crystallizer_salt
   ✅ Success - Duration: 0.00s

📊 Simulation Summary:
   • Successful simulations: 5/5
   • Failed simulations: 0/5
💾 Saved 5 DWSIM results to disk
📊 DWSIM Statistics:
   • Total Simulations: 5
   

Processing files:   0%|          | 0/4 [00:00<?, ?it/s]WARNING:unstructured:libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


 ▶ Processing: Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.docx
   • Success! Saved to: data/02_processed/converted_to_txt/Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.txt
 ▶ Processing: mcp_basics.txt


Processing files:  50%|█████     | 2/4 [00:00<00:00,  3.58it/s]WARNING:unstructured:libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


   • Success! Saved to: data/02_processed/converted_to_txt/mcp_basics.txt
 ▶ Processing: feasibility_factors.txt
   • Success! Saved to: data/02_processed/converted_to_txt/feasibility_factors.txt
 ▶ Processing: Bist_Madan.pdf


Processing files: 100%|██████████| 4/4 [00:01<00:00,  3.19it/s]

   • Success! Saved to: data/02_processed/converted_to_txt/Bist_Madan.txt

 All files processed.

Step 2: Scraping Wikipedia articles...
🔍 Starting Wikipedia article search for 5 keywords...
▶️  Searching for: modular design


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_modular_design.txt
▶️  Searching for: software architecture
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_software_architecture.txt
▶️  Searching for: system design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_system_design.txt
▶️  Searching for: industrial design
✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_industrial_design.txt
▶️  Searching for: supply chain


INFO:pynucleus.integration.dwsim_data_integrator:DWSIMDataIntegrator initialized with output: data/05_output
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


✅  Saved article to: /Users/mohammadalmusaiteer/PyNucleus-Model/data/01_raw/web_sources/wikipedia_supply_chain.txt

✨ Article scraping complete!

Step 3: Processing and chunking documents...
📰 Found 5 Wikipedia articles
📄 Found 4 converted documents
📋 Total documents loaded: 9
✂️ Split into 440 chunks


Step 4: Integrating DWSIM simulation data...
   📊 Using provided DWSIM data: 5 simulations
   ✅ Created 5 simulation chunks
   📊 DWSIM data integrated successfully

🔍 Enhanced Query Capabilities:
   1. What are the performance metrics for the distillation simulation?
   2. How do the reactor conversion rates compare across simulations?
   3. Which simulation showed the highest efficiency?
   📊 Combined 445 total chunks (documents + simulations)
   🔗 Simulation chunks properly formatted as Document objects
   💾 Saving combined chunk data...
   ✅ Chunk data saved successfully

Step 5: Building FAISS vector store...
=== FAISS VectorDB Analysis ===
Started: 2025-06-13 21:34:29
   🔄 Loading 

In [3]:
# Cell 3: Refresh Pipeline (Run this if you encounter errors)
# ===================================
# This cell refreshes the pipeline instance with all latest fixes

print("🔄 Refreshing Pipeline Instance...")

try:
    # Import the latest version
    from src.pynucleus.pipeline.pipeline_utils import PipelineUtils
    
    # Create a fresh pipeline instance with all fixes
    pipeline = PipelineUtils()
    
    print("✅ Pipeline refreshed successfully!")
    print("📊 Now you can run Cell 3 to view the dashboard")
    
    # Quick test to verify it's working
    status = pipeline.quick_test()
    print(f"\n🧪 Quick Test Results:")
    print(f"   📁 Results Directory: {status.get('results_dir', 'Not found')}")
    print(f"   📄 CSV Files: {status.get('csv_files_count', 0)}")
    print(f"   🔗 Integration: {'✅ Enabled' if status.get('integration_enabled', False) else '⚪ Documents only'}")
    print(f"   📊 Total Chunks: {status.get('rag_chunks', 0)}")
    
except Exception as e:
    print(f"❌ Error refreshing pipeline: {e}")
    print("💡 Try restarting the kernel and running all cells from the beginning")


🔄 Refreshing Pipeline Instance...
🔧 Setting up RAG imports...
✅ RAG imports ready!
📂 Loaded 5 existing DWSIM results from disk
🔧 Setting up DWSIM imports...
✅ DWSIM modules imported successfully
📁 Results directory: data/05_output/results
🔧 Pipeline Utils initialized with results dir: data/05_output/results
🔗 DWSIM-RAG integration enabled
✅ Pipeline refreshed successfully!
📊 Now you can run Cell 3 to view the dashboard
🧪 Quick Pipeline Test
------------------------------
📚 RAG: 445 chunks available
🔗 Integration: ⚪ Documents only
📊 DWSIM Statistics:
   • Total Simulations: 5
   • Success Rate: 100.0%
   • Average Duration: 0.00s
🔬 DWSIM: 5 simulations
📁 Output: 2 CSV files

🧪 Quick Test Results:
   📁 Results Directory: data/05_output/results
   📄 CSV Files: 2
   🔗 Integration: ⚪ Documents only
   📊 Total Chunks: 445


In [4]:
# Cell 4: View Results and Summary
# ===================================
# This cell displays your results and provides access to files

print("📊 PyNucleus Results Dashboard")
print("=" * 40)

try:
    # Quick status check
    status = pipeline.quick_test()
    
    print(f"📁 Results Directory: {status['results_dir']}")
    print(f"📄 CSV Files Found: {status['csv_files_count']}")
    
    if status['csv_files_count'] > 0:
        print("\n📋 Available Files:")
        for file_info in status['csv_files']:
            print(f"   • {file_info['name']} ({file_info['size']} bytes)")
    
    # Display detailed summary
    print("\n" + "=" * 40)
    pipeline.view_results_summary()
    
    print("\n🔧 Additional Options:")
    print("   • Re-run Cell 2 to generate new results")
    print("   • Check data/05_output/ folder for all generated files")
    print("   • View Developer_Notebook.ipynb for advanced features")
    
except Exception as e:
    print(f"❌ Error viewing results: {e}")
    print("💡 Please run Cell 2 first to generate results")

📊 PyNucleus Results Dashboard
🧪 Quick Pipeline Test
------------------------------
📚 RAG: 445 chunks available
🔗 Integration: ⚪ Documents only
📊 DWSIM Statistics:
   • Total Simulations: 5
   • Success Rate: 100.0%
   • Average Duration: 0.00s
🔬 DWSIM: 5 simulations
📁 Output: 2 CSV files
📁 Results Directory: data/05_output/results
📄 CSV Files Found: 2

📋 Available Files:
   • dwsim_summary.csv (360 bytes)
   • dwsim_simulation_results.csv (910 bytes)


📊 Pipeline Results Summary
📊 DWSIM Statistics:
   • Total Simulations: 5
   • Success Rate: 100.0%
   • Average Duration: 0.00s
📚 RAG Knowledge Base:
   Total Chunks: 445
   Document Sources: 14
   Integration: ⚪ Documents only
   Avg Chunk Size: 760.5 chars

🔬 DWSIM Simulations:
   Total Simulations: 5
   Success Rate: 100.0%
   Avg Duration: 0.00s

📁 Output Files:
   • dwsim_summary.csv (0.4 KB)
   • dwsim_simulation_results.csv (0.9 KB)

🔧 Additional Options:
   • Re-run Cell 2 to generate new results
   • Check data/05_output/ folder

In [5]:
# # Optional Cell 4A: Run Only Document Analysis (RAG)
# # ====================================================
# # Uncomment and run this cell if you only want document processing

# print("📚 Running Document Analysis Only...")
# rag_results = pipeline.run_rag_only()
# if rag_results:
#     print(f"✅ Processed {len(rag_results['rag_data'])} document queries")
#     print("📁 Results saved to data/05_output/results/")
# else:
#     print("❌ Document analysis failed")


In [6]:
# # Optional Cell 4B: Run Only Chemical Simulations (DWSIM)
# # =======================================================
# # Uncomment and run this cell if you only want DWSIM simulations

# print("🔬 Running Chemical Simulations Only...")
# dwsim_results = pipeline.run_dwsim_only()
# if dwsim_results:
#     print(f"✅ Completed {len(dwsim_results['dwsim_data'])} simulations")
#     print("📁 Results saved to data/05_output/results/")
# else:
#     print("❌ Chemical simulations failed")


In [7]:
# # Optional Cell 5: Clean Up Results
# # =================================
# # Uncomment and run this cell to clear all previous results

# print("🗑️ Cleaning up previous results...")
# pipeline.clean_all_results()
# print("✅ All results cleared. You can now run a fresh analysis.")


In [8]:
# # ========================================
# # VERSION CONTROL (Optional - For Maintainers Only)
# # ========================================
# # Uncomment the lines below if you need to update the repository:

# from datetime import datetime

# # Log end time
# with open("update_log.txt", "a") as f:
#     f.write(f"\n {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} changes made and pushed to origin main\n")

# # Simple GitHub update function
# def update_github():
#     print(" Starting GitHub update...")
#     !git add .
#     print(" Files added to staging")
#     !git commit -m "Update: $(date +'%Y-%m-%d %H:%M:%S')"
#     print(" Changes committed")
#     !git push origin main
#     print(" Changes pushed to GitHub successfully!")

# # To use it, just run:
# update_github()